In [16]:
import geopandas as gpd
import os

# === BASE DIRECTORY ===
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# === FILE PATHS ===
roads_path = os.path.join(base_dir, "street_network.shp")

# === LOAD SHAPEFILE ===
roads = gpd.read_file(roads_path)

In [17]:
import math

roads['qDirLanes2'] = None

def calculate_qDirLanes2(row):
    if row['StOperNEU'] == 1:
        if row['Lanes_OW'] >= 2:
            return row['Lanes_OW']
        else:
            return 1
    elif row['StOperNEU'] == 11:
            return row['NUMLANE']
    elif row['FEDERALFUN'] == 7:
        if row['Has_CL'] == 1:
            return math.ceil(max(row['NUMLANE'] / 2, 1))
        else:
            return 0
    elif row['No_CL'] == 1:
        return 0
    else:
        return math.ceil(max(row['NUMLANE'] / 2, 1))

roads['qDirLanes2'] = roads.apply(calculate_qDirLanes2, axis=1)

In [18]:
roads.loc[(roads['qDirLanes2'] == 0) & (roads['qDirLanes'] == 2), 'qDirLanes2'] = roads['qDirLanes']
roads.loc[(roads['qDirLanes2'] == 0) & (roads['qDirLanes'] == 3), 'qDirLanes2'] = roads['qDirLanes']
roads.loc[(roads['qDirLanes2'] == 4) & (roads['qDirLanes'] == 1), 'qDirLanes2'] = roads['qDirLanes']

In [19]:
# === SAVE THE UPDATED ROADS SHAPEFILE ===
roads.to_file(roads_path, driver="ESRI Shapefile")

In [20]:
# === COMPARE COLUMNS AND GENERATE SUMMARY ===
def compare_qDirLanes(row):
    if row['qDirLanes'] == row['qDirLanes2']:
        return 'Same'
    else:
        return f"{row['qDirLanes']} in qDirLanes vs {row['qDirLanes2']} in qDirLanes2"

# Add a comparison column
roads['Comparison'] = roads.apply(compare_qDirLanes, axis=1)

# Count summary: Group by unique differences and calculate counts
summary_table = roads.groupby('Comparison').size().reset_index(name='Count')

# Output the table
print("Summary Table:")
summary_table

Summary Table:


,Comparison,Count
0,0 in qDirLanes vs 1 in qDirLanes2,259
1,1 in qDirLanes vs 0 in qDirLanes2,467
2,1 in qDirLanes vs 2 in qDirLanes2,2014
3,1 in qDirLanes vs 3 in qDirLanes2,338
4,1 in qDirLanes vs 9 in qDirLanes2,4
5,2 in qDirLanes vs 1 in qDirLanes2,244
6,2 in qDirLanes vs 3 in qDirLanes2,7
7,3 in qDirLanes vs 2 in qDirLanes2,6
8,4 in qDirLanes vs 0 in qDirLanes2,3
9,5 in qDirLanes vs 9 in qDirLanes2,2
